# Yield Modeling Group Projects
Dmitry Savransky (Cornell) and Rhonda Morgan (JPL)

Run the [SSW2024_YieldModelingTutorial1_Setup](https://colab.research.google.com/github/dsavransky/SSWYieldModelingTutorial/blob/main/Notebooks/SSW2024_YieldModelingTutorial1_Setup.ipynb) notebook to download the data. The setup notebook needs to just be run **once** once. If you have already run it for Hands-on Session IV, you do not need to run it again.

## Google Colab Usage
*Please read (don't just hit run) the information given above each code cell as there are separate install cells for Colab*
&#128992;
*and running Python on your computer*
&#128309;.

**Confirm login account**
* Please make sure to be logged in with the Google account you want to use for the exercises before running the code cells below. You can check by clicking the circular account icon in the top right corner of the colab notebook.

**Working directory**
* Note: The software and data will be installed in a directory called "SSW2024/SSWYieldModelingTutorial" in your Google drive. This directory will be created if it does not exist.

**Running cells**
* Run cells individually by clicking on the triangle on each cell

**To Restart runtime**
*   Click on Runtime menu item
*   Select Restart session
*   Select Run code cells individually from the top

**To Recreate runtime**
*   Click on Runtime menu item
*   Select Disconnect and Delete runtime
*   Select Run code cells individually from the top

**To Exit:**
*   Close the browser window

# How to Use This Notebook

This notebook includes descriptions for two hands-on projects based on the content of the Yield Modeling Tutorial.  For both projects, you must first run the setup cells below. The two projects are independent of one another.  You can do one or the other, or try both, in any order.  If you are feeling very ambitious, you can even try combining the two.

## &#128992; Setup Google Drive directory

#### &#128992; **Run the next cell to mount the Google Drive**

In [ ]:
# You will be prompted to Permit this notebook to access your Google Drive files - Click on "Connect to Google Drive"
# You will then be prompted to Choose an account - click on your preferred Google account
# You will then confirm that Google Drive for desktop wants to access your Google Account - scroll to click "Continue"
# You may get another prompt to allow additional access for this to work - scroll to click "Continue"

from google.colab import drive
drive.mount('/content/drive')

#### &#128992; **Run the next cell to define the Yield_Modeling directory that was created in the Yield Modeling Setup Noteobook**

In [ ]:
# This cell should *only* be executed if running the notebook in Google Colab
# SSW Yield Modeling Tutorial
ym_dir = 'SSW2024/Yield_Modeling' #@param {type:"string"}

#### &#128992; **Run the next cell to change to the YieldModelingTutorial directory and install the tutorial and then change directory back to Yield_Modeling**

In [ ]:
# This cell should *only* be executed if running the notebook in Google Colab
# Create the YMT_dir directory in drive
import os

# Google top level drive dir
drive_dir = "/content/drive/MyDrive/"

# YM_dir directory path
ym_path = os.path.join(drive_dir, ym_dir)

# YMT_dir
YMT_dir = "SSWYieldModelingTutorial"

# YMT_path
YMT_path = os.path.join(ym_path, YMT_dir)

# Change to the YMT_path
os.chdir(YMT_path)

# Install the tutorial backend and requirements - this can also take a little while
!pip install -e .

# Refresh package list to pick up new installations
import site
site.main()

## &#128992; Import jupyter widget for Colab

**Run the next cell**

In [ ]:
# need to import third party jupyter widget
from google.colab import output
output.enable_custom_widget_manager()

#  &#128992;  &#128309;  The rest of the notebook is for Colab or Python

In [ ]:
# All users should execute all cells starting with this one
# Ensure that you are working in the Notebooks directory of the tutorial
import os
import pkgutil
codedir = os.path.split(pkgutil.get_loader("SSWYieldModelingTutorial").get_filename())[0]
notebookdir = os.path.abspath(os.path.join(codedir, '..', 'Notebooks'))
assert os.path.exists(notebookdir), "Tutorial Notebooks directory not found."
os.chdir(notebookdir)

# Tutorial Setup

In [ ]:
# This is the start of the tutorial.
# import all required modules
# add any of your own imports here
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
from SSWYieldModelingTutorial import SSWYieldModelingTutorial

In [ ]:
# set up plotting
%matplotlib widget
plt.rcParams.update({'figure.max_open_warning': 0})

# Project 1: Time Allocation Optimization

In this project, our goal is to maximize the summed completeness of an observing program by allocating integration times to some subset of our target list.  The formal problem statement is as follows: given a set of $N$ targets, a maximum total available time $t_\mathrm{max}$, and an overhead time rate of $f_\mathrm{overhead}$ (this is the fraction of the time you spend integrating on each target that goes towards overhead activities and cannot be counted towards your science integration time), find a set of integration times $\{t_i\}_{i=1}^N$ that maximizes the accumulated total completeness, such that the sum of all integration times, setup times, and overheads is less than or equal to $t_\mathrm{max}$.  Note that while we are nominally assigning integration times to all of the targets in our target list, some of these may be zero (meaning that we do not observe those targets at all).  The overhead times apply *only* to those targets with non-zero integration times.   

The mathematical version of the problem statement is:
$$\arg \max_{\{t_i\}} \left(\sum_{i}^n c_i(t_i)\right)$$ 
subject to:
$$t_\mathrm{max} - \sum_{i}^n t_i\left(1+f_\mathrm{overhead}\right) \ge 0 $$

For our input parameters, we will use many of the building blocks developed in the tutorial. 

1. We will use the HWO mission star list as our initial target list:

In [ ]:
targets = SSWYieldModelingTutorial.load_HWO_MissionStars()

2. We will assume the same Earth-like planet population and use the pre-computed completeness PDF: 

In [ ]:
Cpdf, sax, dMagax = SSWYieldModelingTutorial.load_precomputed_completeness()

3. We will use the same (final) coronagraph and observatory description and assumptions about local and exozodi brightness:

In [ ]:
# we'll use a slightly more realistic coronagraph and observatory this time
static_params = {"lam": 550*u.nm, # 550 nm central wavelength
                 "deltaLam": 110*u.nm, # 20% bandpass
                 "D": 6*u.m, # 6 meter telescope
                 "obsc": 0.1, # Primary is 10% obscured
                 "tau": 0.1, # The non-coronagraphic throughput
                 "QE": 0.9, # 90% Quantum Efficiency
                 # Zero-mag flux in photons cm^-2 nm^-1 s^-1 (photons omitted from unit):
                 "F0": 12000/u.cm**2/u.nm/u.s, # Zero-magnitude flux (approximate)
                 "pixelScale": 0.002*u.arcsec, # instantaneous field of view of each detector pixel
                 # dark current in counts/second/pixel (counts and pixels ommited from unit):
                 "darkCurrent": 0.001/u.s,
                 "readNoise": 1e-6, # read noise in electrons/pixel/read
                 "texp": 100*u.s, # single exposure time
                 "ppFac": 0.1, # post-processing factor
                }

coronagraph = {"tau_core": 0.1, # point source throughput
               "tau_occ": 0.2,  # extended source throughput
               "contrast": 1e-10, # contrast (this represents a pretty good coronagraph)
              }

# but now we'll do all the targets at once
target = {"mag_star":  targets['sy_vmag'].values, # apparent magnitude
          "zodi": 23, # local zodi (units omitted as we're using this as an exponent)
          "exozodi": 22, # exozodi (assume a bit brighter than local zodi)
         }

4. Finally, we will assume a maximum total time of 1 year, an overhead rate of 10%, and a target SNR of 5:

In [ ]:
t_max = 1*u.yr
f_overhead = 0.1
SNR = 5

You may use any of the pre-written code in `SSWYieldModelingTutorial`, any of your own code from executing the tutorial, and any additional code you may need.  A few hints.  If using the method `SSWYieldModelingTutorial.Cp_Cb_M` note that the final input (the targeted limiting $\Delta\mathrm{mag}$) can be either a scalar value (the way we used it in the tutorial) or it can be an array of different values (in the latter case, the array must be of the same size as the `mag_star` parameter in the `target` dictionary input.  This allows you to compute integration times for your entire target list at the same time, with different $\Delta\mathrm{mag}$ values for each target. 

A few hints on possible strategies for solving this problem.  There has already been a lot of exploration of this in the literature. One approach, known as AYO or Altruistic Yield Optimization, seeks to remove integration time from one target and assign it to another in such a way as to produce a net increase in completeness.  The methodology is discussed in detail in this paper: https://arxiv.org/pdf/1409.5128 (see section 5.4.2).  A similar, but slightly different formulation, can be found in this paper: https://www.spiedigitallibrary.org/journals/Journal-of-Astronomical-Telescopes-Instruments-and-Systems/volume-6/issue-02/027001/Optimal-scheduling-of-exoplanet-direct-imaging-single-visit-observations-of/10.1117/1.JATIS.6.2.027001.full#_=_  (see section 2.4).

You may find the optimization module of the Python `scipy` package useful here (https://docs.scipy.org/doc/scipy/reference/optimize.html).  Note that most of the methods available there refer to minimization whereas we formulated our problem as a mazimization. That's okay - maximization is the same as minimizing the negative of your objective function.

Most importantly - you will want to figure out a systematic way of deciding which of your targets get observed, and which do not.  The constraints set up by the problem statement make it such that not all targets will be worth looking at. Be sure to visualize the data as you work on this - plots can be incredibly helpful in figuring out what is going on, and how changes in integration time impact changes in completeness.

# Project 2: Observing Schedule

In this project, our goal is to figure out when (and whether) we can schedule a set of planned observations to be consistent with an observatory's keepout map. The formal problem statement is as follows: given a set of $N$ targets, a maximum mission duration $t_\mathrm{max}$, and a pre-computed set of integration times $\{t_i\}_{i=1}^N$ find an observing schedule consistent with a pre-computed keepout map. That is, for each target, find an observation start time $t_i^\mathrm{start}$ such that the interval $[t_i^\mathrm{start}, t_i^\mathrm{start} +t_i]$ is fully available for that target in the keepout map.  The observing schedule cannot contain simultaneous observations of multiple targets (that is, there cannot be any overlaps in any of the observing intervals). Your algorithm should also identify any targets for which an observation is impossible (i.e., there is never an available time window long enough to accommodate a full observation for that target).

For our input parameters, we will use many of the building blocks developed in the tutorial.

1. We will use the HWO mission star list as our initial target list:

In [ ]:
targets = SSWYieldModelingTutorial.load_HWO_MissionStars()

2. We will use the same (final) coronagraph and observatory description and assumptions about local and exozodi brightness to compute our integration times (to a fixed $\Delta \mathrm{mag}$ of 25 for each target):

In [ ]:
# we'll use a slightly more realistic coronagraph and observatory this time
static_params = {"lam": 550*u.nm, # 550 nm central wavelength
                 "deltaLam": 110*u.nm, # 20% bandpass
                 "D": 6*u.m, # 6 meter telescope
                 "obsc": 0.1, # Primary is 10% obscured
                 "tau": 0.1, # The non-coronagraphic throughput
                 "QE": 0.9, # 90% Quantum Efficiency
                 # Zero-mag flux in photons cm^-2 nm^-1 s^-1 (photons omitted from unit):
                 "F0": 12000/u.cm**2/u.nm/u.s, # Zero-magnitude flux (approximate)
                 "pixelScale": 0.002*u.arcsec, # instantaneous field of view of each detector pixel
                 # dark current in counts/second/pixel (counts and pixels ommited from unit):
                 "darkCurrent": 0.001/u.s,
                 "readNoise": 1e-6, # read noise in electrons/pixel/read
                 "texp": 100*u.s, # single exposure time
                 "ppFac": 0.1, # post-processing factor
                }

coronagraph = {"tau_core": 0.1, # point source throughput
               "tau_occ": 0.2,  # extended source throughput
               "contrast": 1e-10, # contrast (this represents a pretty good coronagraph)
              }

# but now we'll do all the targets at once
target = {"mag_star":  targets['sy_vmag'].values, # apparent magnitude
          "zodi": 23, # local zodi (units omitted as we're using this as an exponent)
          "exozodi": 22, # exozodi (assume a bit brighter than local zodi)
         }

C_p, C_b, M = SSWYieldModelingTutorial.Cp_Cb_M(static_params, coronagraph, target, 25)
# total integration times for each target, including 10% overhead:
t_i = SSWYieldModelingTutorial.calc_intTime(C_p, C_b, M, 5)*1.1

3. We will use the same pre-computed keepout-map:

In [ ]:
koTimes, koMap, targNames = SSWYieldModelingTutorial.load_HWO_MissionStars_koMap()

4. Finally, we will assume a total mission time of 5 years.  Note that our keepout map is only defined for 1 year.  You may assume that the target availability is exactly the same in the subsequent four years (i.e., the keepout map just repeats itself over and over again). 

You may use any of the pre-written code in `SSWYieldModelingTutorial`, any of your own code from executing the tutorial, and any additional code you may need.  

A few hints on possible strategies for solving this problem.  There has already been a lot of exploration of this in the literature, much of it focused on various forms of what is known as the 'Traveling Salesman Problem' or TSP. One methodology is described in this paper: https://arxiv.org/pdf/0903.4915 (see section 2.3). Unsurprisingly, this specific problem is an example of a much broader class of scheduling problems.  For some discussion of one modern methodology for scheduling problem solving, see here: https://developers.google.com/optimization/scheduling

Most importantly - you will want to initially determine which targets are fundamentally unobservable by the problem setup and constraints. For the remainder, you will need to figure out a systematic method for assigning observation starting times such that you can fit as many targets as possible into the five year span.  The constraints set up by the problem statement make it such that not every target is observable within five years. Be sure to visualize the data as you work on this - plots can be incredibly helpful in figuring out what your proposed schedule looks like, and how it interacts with the keepout map.

# Bonus: Combine Projects 1 and 2

Can you update your time allocation optimization from project one to be aware of the keepout constraints from project 2?  If so, we may have a job for you.